In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sunil kumar\\OneDrive - AlgoDox Technology PVT LTD\\Desktop\\dl_ops_champios\\vinipro\\reseach'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sunil kumar\\OneDrive - AlgoDox Technology PVT LTD\\Desktop\\dl_ops_champios\\vinipro'

In [5]:
# Data class it import a python tools called data class or no need to manually packge , ceate class 
# that only store data , It reduces the amout of code your need to write 
# no need to write __init__ method , __repr__ method , __eq__ method



# pathlib is impor tthe path class , which helps to your work with file and folder paths to eas;u amd in aclean way for al operating system.


# @dataclass(frozen=True) what is this is a decorator that tells paython that one time fixed that cant chagne the values , ones the obkec is created . it makes the configration to sage from accidental chagne , to Protected if some issues

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True) # if any misstake care
class DataIngestiionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [7]:
from src.cnnClassifier.components import *
from src.cnnClassifier.utils.common import read_yaml, create_directories


ImportError: cannot import name 'read_yaml' from 'src.cnnClassifier.utils.common' (c:\Users\sunil kumar\OneDrive - AlgoDox Technology PVT LTD\Desktop\dl_ops_champios\vinipro\src\cnnClassifier\utils\common.py)

In [ ]:
class ConfigurationManager:
    def __inti__(
        self,
        config_filepath : CONFIG_FILE_PATH,
        params_filepath : PARAMS_FILE_PATH
         
    )-> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) ->None:
        config= self.config.data_ingestion
        
        